In [5]:
import pandas as pd
import allel
import os

In [6]:
strain=['A_J','AKR_J','BALB_CJ','CBA_J','C3H_HeJ','DBA_2J','LP_J']

tools=['BioGraph*',
'breakdancer',
'clever',
'delly',
'gasv',
'gridss',
'indelminer',
'lumpexpress',
'mistrvar',
'pindel',
'popdel',
'rdxplorer',
'smoove',
'sniffles']

In [7]:
df_10=pd.read_csv('../analysis_files/df_10.csv')
df_100=pd.read_csv('../analysis_files/df_100.csv')
df_1000=pd.read_csv('../analysis_files/df_1000.csv')
df_10000=pd.read_csv('../analysis_files/df_10000.csv')

df_10_nondel=pd.read_csv('../analysis_files/df_10_nondel.csv')
df_100_nondel=pd.read_csv('../analysis_files/df_100_nondel.csv')
df_1000_nondel=pd.read_csv('../analysis_files/df_1000_nondel.csv')
df_10000_nondel=pd.read_csv('../analysis_files/df_10000_nondel.csv')

pdList = [df_10,df_100,df_1000,df_10000,df_10_nondel,df_100_nondel,df_1000_nondel,df_10000_nondel] 
df_cat = pd.concat(pdList)

gs_=pd.read_csv('../analysis_files/gs_')

In [8]:
df_cat.head()
df_cat = df_cat[df_cat['length'] >= 50]

In [9]:
df_cat.loc[(df_cat['length'] >= 50) & (df_cat['length'] < 100),'len'] = '50-100'
df_cat.loc[(df_cat['length'] >= 100) & (df_cat['length'] < 500),'len'] = '100-500'
df_cat.loc[(df_cat['length'] >= 500) & (df_cat['length'] < 1000),'len'] = '500-1000'
df_cat.loc[(df_cat['length'] >= 1000),'len'] = '1000+'
df_cat

,Unnamed: 0,tool,strain,length,flag,position,threshold,len
0,0,sniffles,LP_J,1725,FP,6055203,10,1000+
1,1,sniffles,LP_J,34582728,FP,17098523,10,1000+
3,3,sniffles,LP_J,114,FP,17231867,10,100-500
7,7,sniffles,LP_J,2168,FP,18195112,10,1000+
9,9,sniffles,LP_J,618,FP,18396654,10,500-1000
...,...,...,...,...,...,...,...,...
80598,80598,BioGraph*,A_J,36477,TN,61132293,10000,1000+
80599,80599,BioGraph*,A_J,36477,TN,61132293,10000,1000+
80600,80600,BioGraph*,A_J,8527,TN,61175238,10000,1000+
80601,80601,BioGraph*,A_J,8206,TN,61184017,10000,1000+


In [10]:
df_merge_all_len=pd.merge(df_cat, gs_,on=['strain','len'])
del df_merge_all_len['Unnamed: 0_x']
del df_merge_all_len['Unnamed: 0_y']
df_merge_all_len

,tool,strain,length,flag,position,threshold,len,n_true
0,sniffles,LP_J,1725,FP,6055203,10,1000+,139
1,sniffles,LP_J,34582728,FP,17098523,10,1000+,139
2,sniffles,LP_J,2168,FP,18195112,10,1000+,139
3,sniffles,LP_J,4150,FP,18920153,10,1000+,139
4,sniffles,LP_J,1148,FP,21448557,10,1000+,139
...,...,...,...,...,...,...,...,...
936405,BioGraph*,A_J,36477,TN,61132293,10000,1000+,158
936406,BioGraph*,A_J,36477,TN,61132293,10000,1000+,158
936407,BioGraph*,A_J,8527,TN,61175238,10000,1000+,158
936408,BioGraph*,A_J,8206,TN,61184017,10000,1000+,158


In [11]:
df_count_TP = df_merge_all_len[df_merge_all_len['flag'] == 'TP'].groupby(['tool','threshold','strain','len','n_true'],as_index=False)['flag'].count()
df_count_TP = df_count_TP.rename(columns={"flag": "nTP"})
df_count_TP.shape
df_count_TP.head()


,tool,threshold,strain,len,n_true,nTP
0,BioGraph*,10,AKR_J,100-500,234,19
1,BioGraph*,10,AKR_J,1000+,140,10
2,BioGraph*,10,AKR_J,50-100,37,2
3,BioGraph*,10,AKR_J,500-1000,83,5
4,BioGraph*,10,A_J,100-500,249,29


In [12]:
df_count_FP = df_merge_all_len[df_merge_all_len['flag'] == 'FP'].groupby(['tool','threshold','strain','len','n_true'],as_index=False)['flag'].count()
df_count_FP = df_count_FP.rename(columns={"flag": "nFP"})
df_count_FP.shape
df_count_FP.head()



,tool,threshold,strain,len,n_true,nFP
0,BioGraph*,10,AKR_J,100-500,234,153
1,BioGraph*,10,AKR_J,1000+,140,77
2,BioGraph*,10,AKR_J,50-100,37,132
3,BioGraph*,10,AKR_J,500-1000,83,27
4,BioGraph*,10,A_J,100-500,249,162


In [13]:
df_count_TN = df_merge_all_len[df_merge_all_len['flag'] == 'TN'].groupby(['tool','threshold','strain','len','n_true'],as_index=False)['flag'].count()
df_count_TN = df_count_TN.rename(columns={"flag": "nTN"})
df_count_TN.shape
df_count_TN.head()

,tool,threshold,strain,len,n_true,nTN
0,BioGraph*,10,AKR_J,1000+,140,205
1,BioGraph*,10,AKR_J,500-1000,83,1
2,BioGraph*,10,A_J,1000+,158,220
3,BioGraph*,10,BALB_CJ,1000+,157,235
4,BioGraph*,10,BALB_CJ,500-1000,77,2


In [14]:
#for s in strain:
    #for t in tools:
        #for th in [10,100,1000,10000]:
            #for l in ["50-100","100-500","500-1000","1000+"]:
                #if not (((df_count_TP['tool'] == t) & (df_count_TP['strain'] == s) & (df_count_TP['threshold'] == th) & (df_count_TP['len'] == l)).any()):
                    #df_count_TP = df_count_TP.append({'tool' : t , 'strain' : s, 'len' : l,'threshold' : th,'nTP' : 0} , ignore_index=True)
#df_count_TP.shape
#df_count_TP.head()

In [15]:
df_merged = pd.DataFrame(columns=['tool', 'threshold', 'strain', 'len','nTP', 'nFP','nTN'])

for s in strain:
    for t in tools:
        for th in [10,100,1000,10000]:
            for l in ["50-100","100-500","500-1000","1000+"]:
                #dfObj = dfObj.append({'User_ID': 25, 'UserName': 'Jack', 'Action': 'Login'}, ignore_index=True)
                if (((df_count_TP['tool'] == t) & (df_count_TP['strain'] == s) & (df_count_TP['threshold'] == th) & (df_count_TP['len'] == l)).any()):
                    tp = df_count_TP[(df_count_TP['tool'] == t) & (df_count_TP['strain'] == s) & (df_count_TP['threshold'] == th) & (df_count_TP['len'] == l)]
                    tp = tp['nTP']
                    tp = tp.values[0]
                else:
                    tp = 0
                    
                if (((df_count_FP['tool'] == t) & (df_count_FP['strain'] == s) & (df_count_FP['threshold'] == th) & (df_count_FP['len'] == l)).any()):
                    fp = df_count_FP[(df_count_FP['tool'] == t) & (df_count_FP['strain'] == s) & (df_count_FP['threshold'] == th) & (df_count_FP['len'] == l)]
                    fp = fp['nFP']
                    fp = fp.values[0]
                else:
                    fp = 0
                 
                if (((df_count_TN['tool'] == t) & (df_count_TN['strain'] == s) & (df_count_TN['threshold'] == th) & (df_count_TN['len'] == l)).any()):
                    tn = df_count_TN[(df_count_TN['tool'] == t) & (df_count_TN['strain'] == s) & (df_count_TN['threshold'] == th) & (df_count_TN['len'] == l)]
                    tn = tn['nTN']
                    tn = tn.values[0]
                else: 
                    tn = 0
                
                #print(tp.values[0])
                df_merged = df_merged.append({'tool':t, 'threshold':th, 'strain':s, 'len':l,'nTP':tp, 'nFP':fp,'nTN':tn},ignore_index=True)
#df_count_TP.shape


In [16]:
df_merged

,tool,threshold,strain,len,nTP,nFP,nTN
0,BioGraph*,10,A_J,50-100,2,136,0
1,BioGraph*,10,A_J,100-500,29,162,0
2,BioGraph*,10,A_J,500-1000,3,29,0
3,BioGraph*,10,A_J,1000+,13,94,220
4,BioGraph*,100,A_J,50-100,19,119,0
...,...,...,...,...,...,...,...
1563,sniffles,1000,LP_J,1000+,51,6,369
1564,sniffles,10000,LP_J,50-100,11,8,0
1565,sniffles,10000,LP_J,100-500,60,4,0
1566,sniffles,10000,LP_J,500-1000,8,0,0


In [17]:
#for s in strain:
    #for t in tools:
        #for th in [10,100,1000,10000]:
            #if not (((df_count_FP['tool'] == t) & (df_count_FP['strain'] == s) & (df_count_FP['threshold'] == th)).any()):
                #df_count_FP = df_count_FP.append({'tool' : t , 'strain' : s,'threshold' : th,'nFP' : 0} , ignore_index=True)
#df_count_FP.shape
#df_count_FP.head()

In [18]:
#for s in strain:
    #for t in tools:
        #for th in [10,100,1000,10000]:
            #if not (((df_count_TN['tool'] == t) & (df_count_TN['strain'] == s) & (df_count_TN['threshold'] == th)).any()):
                #df_count_TN = df_count_TN.append({'tool' : t , 'strain' : s,'threshold' : th,'nTN' : 0} , ignore_index=True)
#df_count_TN.shape
#df_count_TN.head()

In [19]:
#df_all_len=pd.merge(df_count_TP, df_count_FP,on=['tool','threshold','strain','len','n_true'],how='outer')
#df_all_len=pd.merge(df_all_len, df_count_TN,on=['tool','threshold','strain','len','n_true'],how='outer')
#df_all_len.fillna(0)
#df_all_len.shape
#df_all_len.shape
#df_all_len.head(10)


In [20]:
df_merged_all=pd.merge(df_merged, df_merge_all_len,on=['strain','len','tool','threshold'])
del df_merged_all['length']
del df_merged_all['flag']
del df_merged_all['position']
df_merged_all

,tool,threshold,strain,len,nTP,nFP,nTN,n_true
0,BioGraph*,10,A_J,50-100,2,136,0,37
1,BioGraph*,10,A_J,50-100,2,136,0,37
2,BioGraph*,10,A_J,50-100,2,136,0,37
3,BioGraph*,10,A_J,50-100,2,136,0,37
4,BioGraph*,10,A_J,50-100,2,136,0,37
...,...,...,...,...,...,...,...,...
936405,sniffles,10000,LP_J,1000+,50,7,382,139
936406,sniffles,10000,LP_J,1000+,50,7,382,139
936407,sniffles,10000,LP_J,1000+,50,7,382,139
936408,sniffles,10000,LP_J,1000+,50,7,382,139


In [21]:
df_merged_all['nFN']=df_merged_all['n_true']-df_merged_all['nTP']
df_merged_all

,tool,threshold,strain,len,nTP,nFP,nTN,n_true,nFN
0,BioGraph*,10,A_J,50-100,2,136,0,37,35
1,BioGraph*,10,A_J,50-100,2,136,0,37,35
2,BioGraph*,10,A_J,50-100,2,136,0,37,35
3,BioGraph*,10,A_J,50-100,2,136,0,37,35
4,BioGraph*,10,A_J,50-100,2,136,0,37,35
...,...,...,...,...,...,...,...,...,...
936405,sniffles,10000,LP_J,1000+,50,7,382,139,89
936406,sniffles,10000,LP_J,1000+,50,7,382,139,89
936407,sniffles,10000,LP_J,1000+,50,7,382,139,89
936408,sniffles,10000,LP_J,1000+,50,7,382,139,89


In [24]:
df_merged_all['sensitivity']=df_merged_all['nTP']/df_merged_all['n_true']
df_merged_all['precision']=df_merged_all['nTP']/(df_merged_all['nTP'] + df_merged_all['nFP'])
try:
    return df_merged_all['nTP']/(df_merged_all['nTP'] + df_merged_all['nFP'])
except ZeroDivisionError:
    return 0                                            
                                             
#df_merged_all['specificity']=df_merged_all['nTN']/(df_merged_all['nTN']+df_merged_all['nFP'])
#df_merged_all['f-score']=2*(df_merged_all['sensitivity']*df_merged_all['precision'])/(df_merged_all['sensitivity']+df_merged_all['precision']+0.00000001)


ZeroDivisionError: division by zero

In [87]:
df_all_len.to_csv('../analysis_files/df_all_len.csv')